# Code Lab for development of `sb4dfritz_test.py`

Just trying things out here. Don't expect to understand anything if you're not me.

## Putting it all together for testing

In [1]:
from sb4dfritz import SmartPlug
from time import sleep
from datetime import datetime, timedelta
import random
from scipy.stats import skewnorm
import threading


def add_network_latency(one_way=True):
    """Simulates network latency by sampling from a skew normal distribution."""
    # get sample from a suitable skew normal distribution
    two_way_latency = skewnorm.rvs(5, loc=0.74, scale=0.1, size=1)[0]
    # determine latency
    if one_way:
        latency = two_way_latency / 2
    else:
        latency = two_way_latency
    # wait a bit
    sleep(latency)


# TODO MAKE THIS CODE LOOK MORE PRETTY
class PowerSimulator:
    """Simulates the power consumption of my coffee machine."""
    def __init__(self):
        self.regions = {
            "idle": (250, 350),
            "mid": (75000, 85000),
            "high": (125000, 135000)
        }

        self.stickiness_by_region = {
            "idle": 0.7,
            "mid": 0.2,
            "high": 0.6,
        }

        self.weights = [0.6, 0.2, 0.2]
        self.region_names = ["idle", "mid", "high"]

        self.current_region = random.choices(
            population=self.region_names,
            weights=self.weights,
            k=1
        )[0]

    def get_current_power(self):
        stickiness = self.stickiness_by_region[self.current_region]
        if random.random() > stickiness:
            # Switch to a new region based on weights
            self.current_region = random.choices(
                population=self.region_names,
                weights=self.weights,
                k=1
            )[0]

        # Generate number in current region
        low, high = self.regions[self.current_region]
        return random.randint(low, high)



class MeasurementSimulator():
    """Simulates the measurement mechanism in FRITZ! smart plugs and produces
    "basic_device_stats" in the fritzconnection dictionary format."""
    
    def __init__(self):
        # simulate connected appliances
        self.appliances:PowerSimulator = PowerSimulator()
        # set measure cycle base time
        self.cycle_base_time = None
        self.reset_cycle_base_time()
        # set device stats
        self.basic_device_stats = None
        self._generate_basic_device_stats()
        # attributes needed for sleep mode
        self.sleeping = False
        self._lock = threading.Lock()
        self._timer = None
        # keep awake after initialization
        self.stay_awake()
    
    # STATUS THIS IS LOOKING GOOD
    def _generate_basic_device_stats(self):
        # generate template
        device_stats = self._generate_basic_device_stats_template()
        # generate sufficient data
        cats = device_stats.keys()
        max_count = max([device_stats[cat]['count'] for cat in cats])
        data = {cat:[] for cat in cats}
        for i in range(max_count):
            new_data = self._generate_measure_data()
            for cat in cats:
                data[cat].append(new_data[cat])
        # fill device_stats with the right amounts of data
        for cat in cats:
            cat_count = device_stats[cat]['count'] 
            device_stats[cat]['data'] = data[cat][:cat_count]
        # pass to the basic_device_stats attribute
        self.basic_device_stats = device_stats

    # TODO THE WAY KEAYS ARE DEALT WITH FEELS CLUMSY
    def _generate_basic_device_stats_template(self):
        # designated dictionary keys
        LEVEL_1_KEYS = ['temperature', 'voltage', 'power', 'energy']
        LEVEL_2_KEYS = ['count', 'grid', 'datatime', 'data']
        # designates values for 'count' and 'grid' in order of LEVEL_1_KEYS
        COUNT_VALUES = [96, 360, 360, 12]
        GRID_VALUES = [900, 10, 10, 2678400]
        # initializing the nested dictionary
        level2_template = {key2:None for key2 in LEVEL_2_KEYS}
        device_stats = {key1:level2_template.copy() for key1 in LEVEL_1_KEYS}
        # adding in the values in level 2
        for idx, category in enumerate(LEVEL_1_KEYS):
            device_stats[category]['count'] = COUNT_VALUES[idx]
            device_stats[category]['grid'] = GRID_VALUES[idx]
            device_stats[category]['datatime'] = self.cycle_base_time.replace(microsecond=0)
            device_stats[category]['data'] = []
        return device_stats
    
    # TODO FIND A MORE REALISTIC WAY TO GENERATE RANDOM DATA
    def _generate_measure_data(self):
        data_categories = ['temperature', 'voltage', 'power', 'energy']
        data = {cat:0 for cat in data_categories}
        data['power'] = self.appliances.get_current_power()
        return data
    
    def send_basic_device_stats(self):
        # wake up on request
        self.stay_awake()
        # reset measure cycle base time if sleeping
        if self.sleeping:
            self.reset_cycle_base_time()
        # update data
        self._update_basic_device_stats()
        # add a bit of latency
        add_network_latency()
        # send current device stats
        return self.basic_device_stats
    
    # TODO TEST THIS !!!
    def _update_basic_device_stats(self):
        device_stats = self.basic_device_stats
        current_time = datetime.now()
        for cat in device_stats.keys():
            old_data = device_stats[cat]['data']
            old_datatime = device_stats[cat]['datatime']
            grid = device_stats[cat]['grid']
            grid_delta = timedelta(seconds=grid)
            # update datatime
            new_datatime = old_datatime
            cycles_passed = 0
            while new_datatime + grid_delta <= current_time:
                cycles_passed += 1
                new_datatime += grid_delta
            device_stats[cat]['datatime'] = new_datatime.replace(microsecond=0)
            # add new data values
            new_data = old_data
            for _ in range(cycles_passed):
                new_value = self._generate_measure_data()[cat]
                new_data = [new_value] + new_data
                new_data = new_data[:-1]
            # update data in device_stats
            device_stats[cat]['data'] = new_data
        # update basic_device_stats
        self.basic_device_stats = device_stats

    
    def reset_cycle_base_time(self):
        # get current time minus one second
        t = datetime.now() + timedelta(seconds=-1)
        # replace the microsecond component with a random value
        t = t.replace(microsecond=random.randint(0,1000000))
        # update the cycle base time
        self.cycle_base_time = t

    def stay_awake(self, awake_time=300):
        if self.sleeping:
            self.reset_cycle_base_time()
        with self._lock:
            self.sleeping = False
            if self._timer:
                self._timer.cancel()
            self._timer = threading.Timer(awake_time, self._go_to_sleep)
            self._timer.daemon = True
            self._timer.start()

    def _go_to_sleep(self):
        with self._lock:
            self.sleeping = True



class HomeAutomationDeviceSimulator:
    """Simulates an instance of HomeAutomationDevice in fritzconnection."""

    def __init__(self, name="Smart Plug Simulator"):
        # attributes explicitly used by SmartPlug
        self.DeviceName = name 
        self.Manufacturer = "SB4D" 
        self.ProductName = "FRITZ! Smart Plug Simulator"
        self.identifier = "12345 1234567"
        # attributes implicitly used by SmartPlug 
        # by way of the methods below
        self._switch_state = True
        self.sensor:MeasurementSimulator = MeasurementSimulator()
    
    def is_switchable(self):
        """Checks whether a switchable device is simulated."""
        # Assuming that all devices are smart plugs...
        return True

    def get_switch_state(self):
        """Check whether the simulated device's power switch 
        is on or off."""
        return self.__switch_state
    
    def set_switch(self,target_state:bool):
        """Sets the power switch state of the simulated device."""
        self._switch_state = target_state
    
    def get_basic_device_stats(self):
        # add a bit of latency
        add_network_latency()
        # send request for device stats
        return self.sensor.send_basic_device_stats()


In [2]:
plug_simulator = HomeAutomationDeviceSimulator()
simulated_plug = SmartPlug(plug_simulator)


In [3]:
simulated_plug.turn_off_when_idle_low_latency()



--------------------------------------------------------------------------------
Optimizing latency for power value readouts...
--------------------------------------------------------------------------------
(0) Power:  799.09 W | Latency:  0.40 s | Response Time: 0.86 s | Offset:  1.000 s | Gap:  4.000
(1) Power:  799.09 W | Latency:  1.86 s | Response Time: 0.86 s | Offset:  1.000 s | Gap:  4.000
(2) Power:    3.29 W | Latency:  0.86 s | Response Time: 0.84 s | Offset:  0.025 s | Gap:  2.050
(3) Power:    3.29 W | Latency: 10.37 s | Response Time: 0.83 s | Offset: -0.462 s | Gap:  1.075
(4) Power:  847.72 W | Latency:  0.59 s | Response Time: 0.78 s | Offset: -0.194 s | Gap:  0.537
(5) Power:    3.11 W | Latency:  0.53 s | Response Time: 0.83 s | Offset: -0.303 s | Gap:  0.319
(6) Power:    3.39 W | Latency:  0.46 s | Response Time: 0.82 s | Offset: -0.358 s | Gap:  0.209
(7) Power:    3.34 W | Latency:  0.45 s | Response Time: 0.83 s | Offset: -0.385 s | Gap:  0.155
(8) Power: 1299

[{'power': 799.09,
  'record time': datetime.datetime(2025, 7, 7, 13, 20, 36),
  'request time': datetime.datetime(2025, 7, 7, 13, 20, 35, 544597),
  'response time': datetime.datetime(2025, 7, 7, 13, 20, 36, 402478),
  'duration': 0.857881,
  'latency': 0.402478,
  'base time': datetime.datetime(2025, 7, 7, 13, 20, 36),
  'offset': 1},
 {'power': 799.09,
  'record time': datetime.datetime(2025, 7, 7, 13, 20, 36),
  'request time': datetime.datetime(2025, 7, 7, 13, 20, 37, 612),
  'response time': datetime.datetime(2025, 7, 7, 13, 20, 37, 857716),
  'duration': 0.857104,
  'latency': 1.857716,
  'base time': datetime.datetime(2025, 7, 7, 13, 20, 37),
  'offset': 1.0},
 {'power': 3.29,
  'record time': datetime.datetime(2025, 7, 7, 13, 20, 46),
  'request time': datetime.datetime(2025, 7, 7, 13, 20, 46, 25679),
  'response time': datetime.datetime(2025, 7, 7, 13, 20, 46, 861126),
  'duration': 0.835447,
  'latency': 0.861126,
  'base time': datetime.datetime(2025, 7, 7, 13, 20, 36, 2500

## The finished bits

`HomeAutomationDeviceSimulator` and network latency are done.
`MeasurementSimulator` still needs works. Loading a template here to avoid errors.

In [4]:
from scipy.stats import skewnorm
from time import sleep

def add_network_latency(one_way=True):
    """Simulates network latency by sampling from a skew normal distribution."""
    # get sample from a suitable skew normal distribution
    two_way_latency = skewnorm.rvs(5, loc=0.74, scale=0.1, size=1)[0]
    # determine latency
    if one_way:
        latency = two_way_latency / 2
    else:
        latency = two_way_latency
    # wait a bit
    sleep(latency)


class MeasurementSimulator():
    
    def __init__(self):
        self.basic_device_stats = None
        self.cycle_base_time = None
        self.sleeping = False
        
    def send_basic_device_stats(self):
        # wake up on request
        self.stay_awake()
        # add a bit of latency
        add_network_latency()
        # send current device stats
        return self.basic_device_stats

    def reset_cycle_base_time(self):
        pass

    def stay_awake(self,awake_time=300):
        pass


class HomeAutomationDeviceSimulator:

    def __init__(self, name="Smart Plug Simulator"):
        # attributes explicitly used by SmartPlug
        self.DeviceName = name 
        self.Manufacturer = "SB4D" 
        self.ProductName = "FRITZ! Smart Plug Simulator"
        self.identifier = "12345 1234567"
        # attributes implicitly used by SmartPlug 
        # by way of the methods below
        self._switch_state = True
        self.sensor:MeasurementSimulator = MeasurementSimulator()
    
    def is_switchable(self):
        """Checks whether a switchable device is simulated."""
        # Assuming that all devices are smart plugs...
        return True

    def get_switch_state(self):
        """Check whether the simulated device's power switch 
        is on or off."""
        return self.__switch_state
    
    def set_switch(self,target_state:bool):
        """Sets the power switch state of the simulated device."""
        self._switch_state = target_state
    
    def get_basic_device_stats(self):
        # add a bit of latency
        add_network_latency()
        # send request for device stats
        return self.sensor.send_basic_device_stats()

# Device stat generation

sketching a mechanism:
* on initialization: generate some data -> add method `_generate_basic_device_stats` 
* on request: update data -> add method `_update_basic_device_stats` 

better mechanism:
* on initialization: 
    * generate some data -> add method `_generate_basic_device_stats` 
        * generate empty data template -> add method `_generate_basic_device_stats_template` 
        * generate data -> add method `_generate_measure_data`
            * should generate a single round of data (for all categories)
            * has to be iterated on initalization according to the stats count per category 

* on request: update data -> add method `_update_basic_device_stats` 
    * generate data using `_generate_measure_data`
    * update data and timestamp in device stats 

In [ ]:
from scipy.stats import skewnorm
from time import sleep
from datetime import datetime, timedelta
import random
import threading



class MeasurementSimulator():
    
    def __init__(self, type="smart_plug"):
        # set device type
        self.type = type
        # set measure cycle base time
        self.cycle_base_time = None
        self.reset_cycle_base_time()
        # set device stats
        self.basic_device_stats = None
        self._generate_basic_device_stats()
        # attributes needed for sleep mode
        self.sleeping = False
        self._lock = threading.Lock()
        self._timer = None
        # keep awake after initialization
        self.stay_awake()
    
    # STATUS THIS IS LOOKING GOOD
    def _generate_basic_device_stats(self):
        # generate template
        device_stats = self._generate_basic_device_stats_template()
        # generate sufficient data
        cats = device_stats.keys()
        max_count = max([device_stats[cat]['count'] for cat in cats])
        data = {cat:[] for cat in cats}
        for i in range(max_count):
            new_data = self._generate_measure_data()
            for cat in cats:
                data[cat].append(new_data[cat])
        # fill device_stats with the right amounts of data
        for cat in cats:
            cat_count = device_stats[cat]['count'] 
            device_stats[cat]['data'] = data[cat][:cat_count]
        # pass to the basic_device_stats attribute
        self.basic_device_stats = device_stats

    # TODO THE WAY KEAYS ARE DEALT WITH FEELS CLUMSY
    def _generate_basic_device_stats_template(self):
        # designated dictionary keys
        LEVEL_1_KEYS = ['temperature', 'voltage', 'power', 'energy']
        LEVEL_2_KEYS = ['count', 'grid', 'datatime', 'data']
        # designates values for 'count' and 'grid' in order of LEVEL_1_KEYS
        COUNT_VALUES = [96, 360, 360, 12]
        GRID_VALUES = [900, 10, 10, 2678400]
        # initializing the nested dictionary
        level2_template = {key2:None for key2 in LEVEL_2_KEYS}
        device_stats = {key1:level2_template.copy() for key1 in LEVEL_1_KEYS}
        # adding in the values in level 2
        for idx, category in enumerate(LEVEL_1_KEYS):
            device_stats[category]['count'] = COUNT_VALUES[idx]
            device_stats[category]['grid'] = GRID_VALUES[idx]
            device_stats[category]['datatime'] = self.cycle_base_time.replace(microsecond=0)
            device_stats[category]['data'] = []
        return device_stats
    
    # TODO FIND A MORE REALISTIC WAY TO GENERATE RANDOM DATA
    def _generate_measure_data(self):
        data_categories = ['temperature', 'voltage', 'power', 'energy']
        data = {cat:0 for cat in data_categories}
        data['power'] = self._generate_data()
        return data
    
    def _generate_power_data(self):
        power_states = ["idle", "mid", "high"]
        # choose a power state
        power_status = random.choices(
            population=power_states,
            weights=[0.6, 0.2, 0.2],
            k=1
        )[0]
        # choose a power value according to the chosen state
        if power_status == "idle":
            return random.randint(250, 350)
        elif power_status == "mid":
            return random.randint(75000, 85000)
        elif power_status == "high":
            return random.randint(125000, 135000)        
    
    def send_basic_device_stats(self):
        # wake up on request
        self.stay_awake()
        # reset measure cycle base time if sleeping
        if self.sleeping:
            self.reset_cycle_base_time()
        # update data
        self._update_basic_device_stats()
        # add a bit of latency
        add_network_latency()
        # send current device stats
        return self.basic_device_stats
    
    # TODO TEST THIS !!!
    def _update_basic_device_stats(self):
        device_stats = self.basic_device_stats
        current_time = datetime.now()
        for cat in device_stats.keys():
            old_data = device_stats[cat]['data']
            old_datatime = device_stats[cat]['datatime']
            grid = device_stats[cat]['grid']
            # update datatime
            new_datatime = old_datatime
            cycles_passed = 0
            while new_datatime <= current_time:
                cycles_passed += 1
                grid_delta = timedelta(seconds=grid)
                new_datatime += grid_delta
            device_stats[cat]['datatime'] = new_datatime.replace(microsecond=0)
            # add new data values
            new_data = old_data
            for _ in range(cycles_passed):
                new_value = self._generate_measure_data()[cat]
                new_data = [new_value] + new_data
                new_data = new_data[:-1]
            # update data in device_stats
            device_stats[cat]['data'] = new_data
        # update basic_device_stats
        self.basic_device_stats = device_stats

    
    def reset_cycle_base_time(self):
        # get current time minus one second
        t = datetime.now() + timedelta(seconds=-1)
        # replace the microsecond component with a random value
        t = t.replace(microsecond=random.randint(0,1000000))
        # update the cycle base time
        self.cycle_base_time = t

    def stay_awake(self, awake_time=300):
        if self.sleeping:
            self.reset_cycle_base_time()
        with self._lock:
            self.sleeping = False
            if self._timer:
                self._timer.cancel()
            self._timer = threading.Timer(awake_time, self._go_to_sleep)
            self._timer.daemon = True
            self._timer.start()

    def _go_to_sleep(self):
        with self._lock:
            self.sleeping = True

sensor = MeasurementSimulator()

for cat in ['temperature', 'voltage', 'power', 'energy']:
    assert len(sensor.basic_device_stats[cat]['data']) == sensor.basic_device_stats[cat]['count']


In [173]:
print(sensor.basic_device_stats['power'])
sensor._update_basic_device_stats()
print(sensor.basic_device_stats['power'])


{'count': 360, 'grid': 10, 'datatime': datetime.datetime(2025, 7, 7, 12, 16, 8), 'data': [316, 254, 80102, 81119, 83702, 77606, 130160, 319, 283, 312, 129687, 319, 350, 304, 125036, 125277, 84348, 81747, 282, 129518, 130373, 262, 131190, 268, 134609, 344, 297, 331, 128909, 134439, 329, 288, 344, 128812, 323, 313, 308, 133111, 131725, 255, 344, 282, 80708, 333, 327, 344, 329, 299, 126389, 126358, 132858, 279, 280, 255, 291, 77645, 83356, 78116, 302, 307, 78074, 254, 295, 307, 270, 269, 282, 258, 349, 342, 80545, 280, 76866, 257, 303, 126386, 313, 302, 309, 130955, 80869, 127038, 251, 78484, 132534, 78456, 126433, 279, 134696, 126838, 263, 84296, 133614, 282, 265, 259, 82948, 131820, 83322, 322, 270, 77739, 255, 349, 267, 288, 278, 76927, 255, 323, 317, 328, 129012, 130308, 330, 80567, 129381, 265, 83450, 344, 75976, 125801, 125953, 264, 276, 330, 261, 316, 315, 300, 82343, 77105, 132265, 75702, 333, 338, 288, 76647, 76283, 348, 297, 126820, 313, 126665, 295, 75693, 292, 129340, 82718, 2

In [ ]:
from datetime import datetime, timedelta
import random
import threading

class MeasurementSimulator():
    
    def __init__(self):
        self.basic_device_stats = None
        self._generate_basic_device_stats()
        # set cycle base time
        self.cycle_base_time = None
        self.reset_cycle_base_time()
        # attributes needed for sleep mode
        self.sleeping = False
        self._lock = threading.Lock()
        self._timer = None
        # keep awake after initialization
        self.stay_awake()

    # TODO figure out how to add reasonable 'data' values
    def _generate_basic_device_stats(self):
        # designated dictionary keys
        LEVEL_1_KEYS = ['temperature', 'voltage', 'power', 'energy']
        LEVEL_2_KEYS = ['count', 'grid', 'datatime', 'data']
        # designates values for 'count' and 'grid' in order of LEVEL_1_KEYS
        COUNT_VALUES = [96, 360, 360, 12]
        GRID_VALUES = [900, 10, 10, 2678400]
        # initializing the nested dictionary
        level2_template = {key2:None for key2 in LEVEL_2_KEYS}
        device_stats = {key1:level2_template.copy() for key1 in LEVEL_1_KEYS}
        # adding in the values in level 2
        for idx, category in enumerate(LEVEL_1_KEYS):
            device_stats[category]['count'] = COUNT_VALUES[idx]
            device_stats[category]['grid'] = GRID_VALUES[idx]
            device_stats[category]['datatime'] = self._get_latest_measure_timestamp()
            device_stats[category]['data'] = self._generate_measuredata(category)
        # updating the basic_device_stats attribute
        self.basic_device_stats = device_stats
    

    # TODO decide what to do with the timestamp
    def _get_latest_measure_timestamp(self):
        return datetime.now().replace(microsecond=0)
    
    # TODO find a reasonable way to generate data
    def _generate_measuredata(self, category):
        return []

    def send_basic_device_stats(self):
        # wake up the device
        self.stay_awake()
        # return the curent device stats
        return self.basic_device_stats
    
    def reset_cycle_base_time(self):
        # get current time minus one second
        t = datetime.now() + timedelta(seconds=-1)
        # replace the microsecond component with a random value
        t = t.replace(microsecond=random.randint(0,1000000))
        # update the cycle base time
        self.cycle_base_time = t

    def _go_to_sleep(self):
        with self._lock:
            self.sleeping = True

    def stay_awake(self, awake_time=300):
        if self.sleeping:
            self.reset_cycle_base_time()
        with self._lock:
            self.sleeping = False
            if self._timer:
                self._timer.cancel()
            self._timer = threading.Timer(awake_time, self._go_to_sleep)
            self._timer.daemon = True
            self._timer.start()


In [ ]:
sensor = MeasurementSimulator()

sensor.send_basic_device_stats()

{'temperature': {'count': 96,
  'grid': 900,
  'datatime': datetime.datetime(2025, 7, 6, 23, 8, 36),
  'data': []},
 'voltage': {'count': 360,
  'grid': 10,
  'datatime': datetime.datetime(2025, 7, 6, 23, 8, 36),
  'data': []},
 'power': {'count': 360,
  'grid': 10,
  'datatime': datetime.datetime(2025, 7, 6, 23, 8, 36),
  'data': []},
 'energy': {'count': 12,
  'grid': 2678400,
  'datatime': datetime.datetime(2025, 7, 6, 23, 8, 36),
  'data': []}}

## Generating random power values

I essentially asked ChatGPT to choose random power values like the ones produced by l'angolo. It gave me this:

```python
import random

class StickyRandomGenerator:
    def __init__(self):
        self.regions = {
            "low": (250, 350),
            "mid": (75000, 85000),
            "high": (125000, 135000)
        }

        self.stickiness_by_region = {
            "low": 0.95,
            "mid": 0.85,
            "high": 0.75
        }

        self.weights = [0.6, 0.2, 0.2]
        self.region_names = ["low", "mid", "high"]

        self.current_region = random.choices(
            population=self.region_names,
            weights=self.weights,
            k=1
        )[0]

    def generate(self):
        stickiness = self.stickiness_by_region[self.current_region]
        if random.random() > stickiness:
            # Switch to a new region based on weights
            self.current_region = random.choices(
                population=self.region_names,
                weights=self.weights,
                k=1
            )[0]

        # Generate number in current region
        low, high = self.regions[self.current_region]
        return random.randint(low, high)

# Example usage
gen = StickyRandomGenerator()
for _ in range(30):
    print(gen.generate())

```

This gave me the idea to add another class for `PowerSimulator` which models the connected appliance.

In [ ]:
import random

# TODO MAKE THIS CODE LOOK MORE PRETTY
class PowerSimulator:
    def __init__(self):
        self.regions = {
            "idle": (250, 350),
            "mid": (75000, 85000),
            "high": (125000, 135000)
        }

        self.stickiness_by_region = {
            "idle": 0.7,
            "mid": 0.2,
            "high": 0.6,
        }

        self.weights = [0.6, 0.2, 0.2]
        self.region_names = ["idle", "mid", "high"]

        self.current_region = random.choices(
            population=self.region_names,
            weights=self.weights,
            k=1
        )[0]

    def generate(self):
        stickiness = self.stickiness_by_region[self.current_region]
        if random.random() > stickiness:
            # Switch to a new region based on weights
            self.current_region = random.choices(
                population=self.region_names,
                weights=self.weights,
                k=1
            )[0]

        # Generate number in current region
        low, high = self.regions[self.current_region]
        return random.randint(low, high)

# Example usage
gen = PowerSimulator()
for _ in range(30):
    print(gen.generate())


284
320
332
271
264
335
287
258
278
262
275
250
286
298
268
306
262
250
328
295
346
323
349
301
132495
130505
129106
81360
125738
278
